In [1]:
import os
from binance import Client
import pandas as pd
import numpy as np
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta


In [2]:
!source .env

In [3]:
%reload_ext dotenv
%dotenv

In [4]:
binance_api = os.environ.get('BINANCE_API', None)
binance_secret = os.environ.get('BINANCE_SECRET', None)

In [5]:
client = Client(binance_api, binance_secret)
client.API_URL = 'https://testnet.binance.vision/api'

In [6]:
exchange_info = pd.DataFrame.from_records(client.get_exchange_info()['symbols'])
coins = exchange_info.baseAsset.unique()

In [7]:
from tqdm import tqdm

def get_crypto_info(client, coin_name, exchange_info, interval='1h', start_date=dt.now()-relativedelta(days=30)
):
  ticker = f'{coin_name.upper()}USDT'
  if ticker not in exchange_info.symbol.values:
    print(ticker, 'not in API')
    return None
  else:
    timestamp = int(start_date.timestamp()*1000)
    bars = client.get_historical_klines(ticker, interval, timestamp)
    if bars:
      hist = pd.DataFrame.from_records([dict(zip(['date', 'open'], bar[:2])) for bar in bars]).set_index('date')
      hist.index = list(map(lambda x: dt.fromtimestamp(x/1000), hist.index))
      return hist
    print(f"Couldn't retrieve data for {coin_name}")
    return None

dfs = [
  get_crypto_info(client, coin, exchange_info)[['open']].rename(columns={'open':coin.upper()})
  for coin in tqdm(coins)
  if get_crypto_info(client, coin, exchange_info) is not None
]

100%|██████████| 6/6 [00:15<00:00,  2.55s/it]


In [8]:
concat_df = pd.concat(dfs,axis=1).astype(np.float128)[2:]
concat_df

,BNB,BTC,ETH,LTC,TRX,XRP
2022-09-07 06:00:00,262.18,18806.51,1521.01,54.37,0.06063,0.3203
2022-09-07 07:00:00,264.60,18789.51,1518.78,54.48,0.06044,0.3218
2022-09-07 08:00:00,263.30,18747.98,1512.56,54.22,0.06025,0.3223
2022-09-07 09:00:00,263.60,18740.17,1513.80,54.18,0.06029,0.3231
2022-09-07 10:00:00,264.30,18773.60,1517.48,54.90,0.06049,0.3243
...,...,...,...,...,...,...
2022-09-28 04:00:00,268.60,18698.36,1277.48,51.69,0.05882,0.4226
2022-09-28 05:00:00,270.50,18782.66,1285.53,52.05,0.05907,0.4239
2022-09-28 06:00:00,269.90,18738.50,1282.26,51.96,0.05912,0.4233
2022-09-28 07:00:00,269.70,18652.04,1273.44,52.01,0.05842,0.4219


In [9]:
from statsmodels.tsa.stattools import coint

all_combinations = [[coins[i], coins[j]] for i in range(len(coins)) for j in range(len(coins)) if i != j and j > i]

p_values = {}
for combination in tqdm(all_combinations):
  p_values['-'.join(combination)] = coint(concat_df.dropna()[combination[0]], concat_df.dropna()[combination[1]])[1]

100%|██████████| 15/15 [00:01<00:00, 10.94it/s]


In [10]:
cointegrations = list(filter(lambda x: x[1] <= 0.05, p_values.items()))

pairs = [pair[0].split('-') for pair in cointegrations]

coins = list(np.unique(np.array(pairs).flatten()))

cointegrations

[('BTC-ETH', 0.028506991701698434), ('BTC-LTC', 0.0415324695364492)]

In [11]:
div_threshold = float(0.5) # std threshold as to when the ratio diverges
conv_threshold = float(0.1) # std threshold as to when the ratio converges
init_threshold = float(0.1) # make initial bet to buy some stock
window = 24*10

print('Diversion threshold is', div_threshold)
print('Conversion threshold is', conv_threshold)
print('Initial threshold is', init_threshold)
print('Window size is', window)

Diversion threshold is 0.5
Conversion threshold is 0.1
Initial threshold is 0.1
Window size is 240


In [12]:
lot_sizes = {}

for coin in coins:
  filters = client.get_symbol_info(symbol=f'{coin}USDT')['filters']
  mkt_lot_size = list(filter(lambda x: 'MARKET_LOT_SIZE' == x['filterType'], filters))
  lot_size = list(filter(lambda x: 'LOT_SIZE' == x['filterType'], filters))
  if not mkt_lot_size:
    if not lot_size:
      max = np.inf
      min = 0
      decimals = 4
    else:
      max = float(lot_size[0]['maxQty'])
      min = float(lot_size[0]['minQty'])
      decimals = lot_size[0]['stepSize'].split('.')[1].find('1') + 1
  else:
    max = float(mkt_lot_size[0]['maxQty'])
    min = float(mkt_lot_size[0]['minQty'])
    decimals = lot_size[0]['stepSize'].split('.')[1].find('1') + 1
  
  lot_sizes[coin] = {
    'max': max,
    'min': min,
    'decimals': decimals
  }

lot_sizes

{'BTC': {'max': 100.0, 'min': 0.0, 'decimals': 6},
 'ETH': {'max': 1000.0, 'min': 0.0, 'decimals': 5},
 'LTC': {'max': 1000.0, 'min': 0.0, 'decimals': 5}}

In [13]:
from sklearn import preprocessing

def normalize(df: pd.DataFrame, method:str='std'):
  if method == 'min_max':
    scaler = preprocessing.MinMaxScaler()
    return pd.DataFrame(scaler.fit_transform(df), columns=df.columns, index=df.index), scaler
  if method == 'pct_cumsum':
    return df.pct_change().cumsum().dropna(), None
  if method == 'pct':
    return df.pct_change().dropna(), None
  if method == 'std':
    scaler = preprocessing.StandardScaler()
    return pd.DataFrame(scaler.fit_transform(df), columns=df.columns, index=df.index), scaler
  if method == 'None':
    return df, None

In [14]:
def make_transaction(side, coin, amount, price, order_type, test):
  func = client.create_test_order if test else client.create_order
  amounts = split_orders(coin, amount)
  decimals = lot_sizes[coin]['decimals']
  for amount in amounts:
    if order_type=='MARKET':
      res = func(
        symbol=f'{coin.upper()}USDT',
        side=side,
        type=order_type,
        quantity=amount,
      )
    elif order_type=='LIMIT':
      res = func(
        symbol=f'{coin.upper()}USDT',
        side=side,
        type=order_type,
        quantity=amount,
        icebergQty=round(amount/5, decimals),
        price=price,
        timeInForce='GTC'
      )
    else:
      raise ValueError(f"Wrong order_type parameter ({order_type}). Options are: 'MARKET' or 'LIMIT'")
    if res != {}:
      if res['status'] != 'FILLED':
        if res['status'] != 'NEW':
          raise RuntimeWarning(f"Couldn't complete order! Status: {res['status']}")
        else:
          # while True:
          #   currentOrder = client.get_order(symbol=res['symbol'],orderId=res['orderId'])
          #   if currentOrder['status']=='FILLED':
          #       print("{}: {} at {}".format(side, amount,price))
          #       break
          pass

def buy_stock(coin, amount, price, test=False):
  try:
    make_transaction('BUY', coin, amount, price, 'MARKET', test)
  except RuntimeWarning:
    make_transaction('BUY', coin, amount, price, 'LIMIT', test)


def sell_stock(coin, amount, price, test=False):
  try:
    make_transaction('SELL', coin, amount, price, 'MARKET', test)
  except RuntimeWarning:
    make_transaction('SELL', coin, amount, price, 'LIMIT', test)

def split_orders(coin, amount):
  max = lot_sizes[coin]['max']
  decimals = lot_sizes[coin]['decimals']
  if amount > max:
    values = [round(max, decimals) for _ in range(int(amount//max))]
    if amount%max != 0: 
      values.append(round(amount%max, decimals))
    return values
  return [amount]


def get_available_cash(base_asset='USDT'):
  return float(client.get_asset_balance(asset=base_asset)['free'])

def get_asset_balance(asset):
  return float(client.get_asset_balance(asset=asset)['free'])

def get_asset_current_price(asset):
  return float(client.get_symbol_ticker(symbol=f'{asset}USDT')['price'])

def get_current_prices(coins):
  time = dt.now().replace(microsecond=0)
  curr_prices_array = pd.DataFrame.from_records(client.get_all_tickers()).values
  curr_prices = {}
  for ticker, price in curr_prices_array:
    if 'USDT' in ticker:
      curr_prices[ticker.rstrip('USDT')] = float(price)
  return pd.DataFrame.from_dict({time: curr_prices}, orient='index')[coins]

In [15]:
test = False

train_df = pd.concat([
                      concat_df[coins][-window:].copy(),
                      get_current_prices(coins)
                    ])

last_ratios = []

for j, pair in enumerate(pairs):
    train_df[f'ratio_{j}'] = train_df[pair[0]]/train_df[pair[1]]
    norm_ratio, scaler = normalize(train_df[[f'ratio_{j}']])
    last_ratios.append(norm_ratio[f'ratio_{j}'][-1])

ratios_over_threshold = list(map(lambda x: abs(x) > div_threshold, last_ratios))
ratios_in_conv_threshold = list(map(lambda x: abs(x) < conv_threshold, last_ratios))

if any(ratios_in_conv_threshold):
  for j, in_th in enumerate(ratios_in_conv_threshold):
    if in_th:
      # sell stocks in pair
      print(f'CONV THRES PAIR: {pairs[j]}')
      for i in range(2):
        stock = pairs[j][i]
        price = get_asset_current_price(stock)
        sell_amount = get_asset_balance(stock)
        print(f'CONV THRES SELL: {sell_amount} of {stock}')
        if sell_amount > 0:
          sell_stock(stock, sell_amount, price, test)


if any(ratios_over_threshold):
  # sell first
  for j, over_th in enumerate(ratios_over_threshold):
    if over_th:
      print(f'OVER THRES PAIR: {pairs[j]}')
      stock = pairs[j][0] if last_ratios[j] > 0 else pairs[j][1]
      price = get_asset_current_price(stock)
      sell_amount = get_asset_balance(stock)
      print(f'OVER THRES SELL: {sell_amount} of {stock}')
      if sell_amount > 0:
        sell_stock(stock, sell_amount, price, test)

  # then buy with weighted prices

  over_list = list(filter(lambda x: abs(x) > div_threshold, last_ratios))

  weights = np.abs(over_list)/sum(np.abs(over_list))

  split = get_available_cash()*weights
  w_idx = 0
  for j, over_th in enumerate(ratios_over_threshold):
    if over_th:
      print(f'OVER THRES PAIR: {pairs[j]}')
      stock = pairs[j][1] if last_ratios[j]>0 else pairs[j][0]
      price = get_asset_current_price(stock)
      buy_amount = round(split[w_idx]/price, lot_sizes[stock]['decimals'])
      print(f'OVER THRES BUY: {buy_amount} of {stock}')
      buy_stock(stock,buy_amount, price, test)
      w_idx+=1

OVER THRES PAIR: ['BTC', 'ETH']


BinanceAPIException: APIError(code=-2015): Invalid API-key, IP, or permissions for action.

In [23]:
pd.DataFrame.from_records(client.get_open_orders())

,symbol,orderId,orderListId,clientOrderId,price,origQty,executedQty,cummulativeQuoteQty,status,timeInForce,type,side,stopPrice,icebergQty,time,updateTime,isWorking,origQuoteOrderQty
0,ETHUSDT,2623679,-1,oy2iKWJQs6EJk2ASfxvq26,2743.00000000,99.97200000,4.15732000,11403.52876000,PARTIALLY_FILLED,GTC,LIMIT,SELL,0.00000000,9.99720000,1651167540434,1651235327199,True,0.00000000
1,XRPUSDT,1485299,-1,OF50cC57G3KvXov5XA0Gkr,0.65400000,10000.00000000,1000.00000000,654.00000000,PARTIALLY_FILLED,GTC,LIMIT,BUY,0.00000000,1000.00000000,1651167544927,1651168436406,True,0.00000000
2,XRPUSDT,1485300,-1,unGpc3LHDD46M815KpcRlo,0.65400000,10000.00000000,1000.00000000,654.00000000,PARTIALLY_FILLED,GTC,LIMIT,BUY,0.00000000,1000.00000000,1651167545219,1651168436406,True,0.00000000
3,XRPUSDT,1485301,-1,D3jvmcgL8GnIZ83QT169yj,0.65400000,10000.00000000,1000.00000000,654.00000000,PARTIALLY_FILLED,GTC,LIMIT,BUY,0.00000000,1000.00000000,1651167545547,1651168436406,True,0.00000000
4,XRPUSDT,1485302,-1,9gmc9WEd02KoPb3R9Z6AI3,0.65400000,10000.00000000,1000.00000000,654.00000000,PARTIALLY_FILLED,GTC,LIMIT,BUY,0.00000000,1000.00000000,1651167545961,1651168436406,True,0.00000000
5,XRPUSDT,1485303,-1,bCGSZi92bCqLNSOWkrcCfL,0.65400000,10000.00000000,1000.00000000,654.00000000,PARTIALLY_FILLED,GTC,LIMIT,BUY,0.00000000,1000.00000000,1651167546368,1651168436406,True,0.00000000
6,XRPUSDT,1485304,-1,IrOfFSBuq5Q3fMMcKuPGuj,0.65400000,10000.00000000,1000.00000000,654.00000000,PARTIALLY_FILLED,GTC,LIMIT,BUY,0.00000000,1000.00000000,1651167546661,1651168443389,True,0.00000000
7,XRPUSDT,1485305,-1,GvqAkTstl8eJubPFHSpZ6a,0.65400000,10000.00000000,1000.00000000,654.00000000,PARTIALLY_FILLED,GTC,LIMIT,BUY,0.00000000,1000.00000000,1651167546987,1651168443389,True,0.00000000
8,XRPUSDT,1485306,-1,p7X8CjnEsKZ0dCgLqfe9lM,0.65400000,10000.00000000,1000.00000000,654.00000000,PARTIALLY_FILLED,GTC,LIMIT,BUY,0.00000000,1000.00000000,1651167547392,1651168443389,True,0.00000000
9,XRPUSDT,1485307,-1,Cj1h85uGeHEF2V0KC4gCqC,0.65400000,10000.00000000,1000.00000000,654.00000000,PARTIALLY_FILLED,GTC,LIMIT,BUY,0.00000000,1000.00000000,1651167547799,1651168443389,True,0.00000000


In [109]:
client.cancel_order(symbol='ETHUSDT', orderId=2623623)

{'symbol': 'ETHUSDT',
 'origClientOrderId': 'S4bXFCKxLUqzqgutvXrJqs',
 'orderId': 2623623,
 'orderListId': -1,
 'clientOrderId': 'c15TZ8jJebBGbnRaazTOs1',
 'price': '2743.00000000',
 'origQty': '99.97200000',
 'executedQty': '0.00000000',
 'cummulativeQuoteQty': '0.00000000',
 'status': 'CANCELED',
 'timeInForce': 'GTC',
 'type': 'LIMIT',
 'side': 'SELL',
 'icebergQty': '9.99720000'}

In [16]:
client.get_account()['balances']

[{'asset': 'BNB', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'BTC', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
 {'asset': 'ETH', 'free': '0.00000000', 'locked': '95.81468000'},
 {'asset': 'LTC', 'free': '1416.21885000', 'locked': '0.00000000'},
 {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
 {'asset': 'USDT', 'free': '15602.23758705', 'locked': '314982.81540000'},
 {'asset': 'XRP', 'free': '90509.80000000', 'locked': '0.00000000'}]

In [141]:
client.get_symbol_ticker(symbol='ETHUSDT')

{'symbol': 'ETHUSDT', 'price': '2734.77000000'}

In [24]:
pd.DataFrame.from_records(client.get_order_book(symbol='ETHBUSD')['asks'])

ReadTimeout: HTTPSConnectionPool(host='testnet.binance.vision', port=443): Read timed out. (read timeout=10)

In [ ]:
pd.DataFrame.from_records(client.get_order_book(symbol='ETHBUSD')['bids'])

""
